In [1]:
from classes import *
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as MplPolygon

import geopandas as gpd
from shapely.geometry import Polygon as ShapelyPolygon
import pandas as pd # Used to Export Data to csv format, so long processing steps only have to be done once
import json

from os import path

In [2]:
data1_path = "data/Polygons_no_mountains.geojson"
data2_path = "data/Polygons_with_mountains.geojson"
data3_path = "data/swissnames_points_json_export.geojson"

json_files = [(data1_path, "Gemeinden_ohne_Berge"),
(data2_path, "Gemeinden_mit_Berge"),
(data3_path, "Swissnames_Points")]


## Read in Data (Marek)

Short description what happens and how

In [3]:
data = process_json_file(json_files)

## Preprocessing (Sebi)

Samesame

In [4]:
polygon_data = Polygon_Data(data['Gemeinden_mit_Berge'],data['Gemeinden_ohne_Berge'])

In [5]:
polygon_data.remove_unique_entries()

10 unique objects removed from municipalities_polygons:
Polygon Zwieselberg consisting of 6 points BFS_Nr=947
Polygon Wangenried consisting of 6 points BFS_Nr=993
Polygon Escholzmatt-Marbach consisting of 23 points BFS_Nr=1010
Polygon Lüterswil-Gächliwil consisting of 9 points BFS_Nr=2456
Polygon Bergün Filisur consisting of 36 points BFS_Nr=3544
Polygon Turgi consisting of 9 points BFS_Nr=4042
Polygon Novaggio consisting of 11 points BFS_Nr=5207
Polygon Törbel consisting of 11 points BFS_Nr=6296
Polygon Beurnevésin consisting of 7 points BFS_Nr=6773
Polygon Bonfol consisting of 9 points BFS_Nr=6775

16 unique objects removed from muns_only_vegetation_area_polygons:
Polygon Valsot consisting of 43 points BFS_Nr=3764
Polygon Ponte Capriasca consisting of 6 points BFS_Nr=5212
Polygon Rüeggisberg consisting of 17 points BFS_Nr=880
Polygon Zuoz consisting of 8 points BFS_Nr=3791
Polygon Samedan consisting of 14 points BFS_Nr=3786
Polygon Madulain consisting of 8 points BFS_Nr=3783
Polygon 

In [6]:
# Polygon Intersection
municipalities_mountain_area = polygon_data.apply_difference()

In [7]:
point_data = data['Swissnames_Points']

In [8]:
# Flatten Lists and remove polygons smaller than 100 m^2 
municipalities_mountain_area_flattened_list = [item for sublist in municipalities_mountain_area for item in sublist if item.area() >= 100]
len(municipalities_mountain_area_flattened_list)

2894

In [9]:
# The function returns a list containing polygon id, point id, point name and point type
# !!!Warning: Long Processing Time!!!


#result_only_veg = point_polygon_matching(point_data,polygon_data.cleaned_mun_only_vegetation_polys)

#result_full_polys = point_polygon_matching(point_data,polygon_data.cleaned_mun_polys)

#result_mountain_polys = point_polygon_matching(point_data,municipalities_mountain_area_flattened_list)


In [10]:

# Save DataFrame to CSV file
#result_only_veg_df = pd.DataFrame(result_only_veg)
#result_only_veg_df.to_csv('data/matched_points_vegetation.csv', index=False)

#result_mountain_polys_df = pd.DataFrame(result_mountain_polys)
#result_mountain_polys_df.to_csv('data/matched_points_mountain.csv', index=False)

#result_full_polys_df = pd.DataFrame(result_full_polys)
#result_full_polys_df.to_csv('data/matched_points_full.csv', index=False)

In [11]:
# Read CSV
vegetation = pd.read_csv('data/matched_points_vegetation.csv')
mountains = pd.read_csv('data/matched_points_mountain.csv')
full = pd.read_csv('data/matched_points_full.csv')

# Rename Columns
vegetation.rename(columns={'0': 'polygon_id', '1': 'point_id', '2': 'name', '3': 'objektart'}, inplace=True)
mountains.rename(columns={'0': 'polygon_id', '1': 'point_id', '2': 'name', '3': 'objektart'}, inplace=True)
full.rename(columns={'0': 'polygon_id', '1': 'point_id', '2': 'name', '3': 'objektart'}, inplace=True)

        
# Grouping and counting for the vegetation dataframe
vegetation_grouped = vegetation.groupby(['polygon_id', 'objektart']).size().reset_index(name='point_count')

# Grouping and counting for the mountains dataframe
mountains_grouped = mountains.groupby(['polygon_id', 'objektart']).size().reset_index(name='point_count')

# Grouping and counting for the full dataframe
full_grouped = full.groupby(['polygon_id', 'objektart']).size().reset_index(name='point_count')


print(vegetation_grouped)

      polygon_id objektart  point_count
0              1  Bergname            2
1              1  Flurname          108
2              2  Bergname            1
3              2  Flurname          193
4              3  Bergname            1
...          ...       ...          ...
3418        6809  Flurname          544
3419        6810  Bergname            5
3420        6810  Flurname          461
3421        6811  Bergname            1
3422        6811  Flurname          142

[3423 rows x 3 columns]


In [12]:
#join the dataframes with the two additional attributes with the polygons via the polygon_id
polygon_data.join_csv(full_grouped, vegetation_grouped, mountains_grouped)

1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12
13
13
14
22
22
23
23
24
24
25
26
26
27
28
28
29
31
31
33
33
34
34
35
35
37
37
38
38
39
39
40
40
41
41
43
43
51
51
52
52
53
53
54
54
55
55
56
56
57
57
58
58
59
59
60
60
61
61
62
62
63
63
64
64
65
65
66
67
67
68
68
69
69
70
70
71
71
72
72
81
82
82
83
83
84
85
85
86
86
87
88
88
89
89
90
90
91
91
92
92
93
94
94
95
95
96
96
97
97
98
98
99
99
100
100
101
101
102
102
111
111
112
112
113
113
114
114
115
115
116
116
117
117
118
118
119
119
120
120
121
121
131
131
135
136
136
137
138
139
139
141
141
151
151
152
152
153
153
154
154
155
156
156
157
158
159
160
160
161
161
172
172
173
173
176
176
177
177
178
178
180
180
181
181
182
182
191
191
192
192
193
194
195
195
196
197
197
198
198
199
199
200
200
211
211
213
213
214
214
215
215
216
216
218
218
219
219
220
220
221
221
223
223
224
224
225
225
226
226
227
227
228
228
230
230
231
231
241
241
242
242
243
243
244
244
245
245
246
246
247
248
248
249
250
250
251
251
261
261
291
291
292
292
293
293

IndexError: list index out of range

In [ ]:
#create a map for both types of polygons (vegetation and mountain areas)
polygon_data.plot_poly_both()

In [ ]:
#create a map plot for the points in polygons with "Flurnamen" objektarten
polygon_data.plot_obj_flur()

In [ ]:
#create a map plot for the points in polygons with "Bergnamen" objektarten
polygon_data.plot_obj_berg()